In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras import Sequential,regularizers
from keras.layers import Dense
from sklearn.metrics import confusion_matrix
import torch
from torch import nn

# Loading data 

In [2]:
data = pd.read_csv("../data/compas-scores-two-years.csv")

# Data processing

## Chose useful rows and columns

In [3]:
data = data.loc[data['race'].isin(['African-American','Caucasian'])]
data=data[[ 'sex','age', 'race', 'priors_count',
             'c_charge_degree', 'c_charge_desc',
             'start', 'end', 'event', 'two_year_recid']]
data = data.dropna()
data.shape

(6129, 10)

In [4]:
data.head()

,sex,age,race,priors_count,c_charge_degree,c_charge_desc,start,end,event,two_year_recid
1,Male,34,African-American,0,F,Felony Battery w/Prior Convict,9,159,1,1
2,Male,24,African-American,4,F,Possession of Cocaine,0,63,0,1
3,Male,23,African-American,1,F,Possession of Cannabis,0,1174,0,0
6,Male,41,Caucasian,14,F,Possession Burglary Tools,5,40,1,1
8,Female,39,Caucasian,0,M,Battery,2,747,0,0


## Label

In [5]:
label = data['two_year_recid']

In [6]:
sf = (data[['race']]=='Caucasian').astype(int)
sf.index = range(sf.shape[0])

## Normalize numerical columns and encode categorical columns

In [7]:
num = data._get_numeric_data()
num = num.drop(labels='two_year_recid',axis=1)
ss = StandardScaler()
num_ss = ss.fit_transform(num)
num = pd.DataFrame(num_ss,columns=num.columns)

In [8]:
num_cols = data._get_numeric_data().columns
cat = data.drop(columns = num_cols, axis = 1)
cat = cat.drop(labels='race',axis=1)
cat = pd.get_dummies(cat)
cat.index = range(cat.shape[0])

In [9]:
clean_data = pd.concat([sf,cat,num], axis=1)

# Model

## Basemodel

### Get train, validation, test sets

In [10]:
X_train, X_test, y_train, y_test = train_test_split(clean_data,label,test_size=1/7)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=1/6)

In [11]:
import matplotlib.pyplot as plt

class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

epochs = 1000
input_dim = 420 
output_dim = 1 
learning_rate = 0.01

model = LogisticRegression(input_dim,output_dim)
criterion = nn.BCELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

X_train, X_test = torch.Tensor(np.array(X_train)),torch.Tensor(np.array(X_test))
y_train, y_test = torch.Tensor(np.array(y_train)),torch.Tensor(np.array(y_test))

losses = []
losses_test = []
Iterations = []
iter = 0
for epoch in range(epochs):
    x = X_train
    labels = y_train
    optimizer.zero_grad() # Setting our stored gradients equal to zero
    outputs = model(X_train)
    loss = criterion(torch.squeeze(outputs), torch.tensor(np.array(labels))) # [200,1] -squeeze-> [200]
    loss.backward() # Computes the gradient of the given tensor w.r.t. graph leaves 
    optimizer.step() # Updates weights and biases with the optimizer (SGD)
    
    iter+=1
    if iter%100==0:
        # calculate Accuracy
        with torch.no_grad():
            # Calculating the loss and accuracy for the test dataset
            correct_test = 0
            total_test = 0
            outputs_test = torch.squeeze(model(X_test))
            loss_test = criterion(outputs_test, torch.tensor(np.array(y_test)))
            
            predicted_test = outputs_test.round().detach().numpy()
            total_test += y_test.size(0)
            correct_test += np.sum(predicted_test == y_test.detach().numpy())
            accuracy_test = 100 * correct_test/total_test
            losses_test.append(loss_test.item())
            
            # Calculating the loss and accuracy for the train dataset
            total = 0
            correct = 0
            total += y_train.size(0)
            correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == y_train.detach().numpy())
            accuracy = 100 * correct/total
            losses.append(loss.item())
            Iterations.append(iter)
            
            print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
            print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")

Iteration: 100. 
Test - Loss: 341.6089172363281. Accuracy: 91.0958904109589
Train -  Loss: 2510.0908203125. Accuracy: 91.61526159469956

Iteration: 200. 
Test - Loss: 348.3466796875. Accuracy: 90.8675799086758
Train -  Loss: 2397.07958984375. Accuracy: 91.70664838930774

Iteration: 300. 
Test - Loss: 354.2894287109375. Accuracy: 90.98173515981735
Train -  Loss: 2307.4599609375. Accuracy: 91.75234178661184

Iteration: 400. 
Test - Loss: 359.0176086425781. Accuracy: 90.8675799086758
Train -  Loss: 2233.6845703125. Accuracy: 91.77518848526388

Iteration: 500. 
Test - Loss: 362.7321472167969. Accuracy: 90.6392694063927
Train -  Loss: 2173.404296875. Accuracy: 91.82088188256797

Iteration: 600. 
Test - Loss: 365.6965637207031. Accuracy: 90.52511415525115
Train -  Loss: 2121.65576171875. Accuracy: 91.84372858122002

Iteration: 700. 
Test - Loss: 368.06170654296875. Accuracy: 90.6392694063927
Train -  Loss: 2078.51416015625. Accuracy: 91.93511537582819

Iteration: 800. 
Test - Loss: 369.85842

## Model with PR

### Get train, validation, test sets

In [34]:
X_train, X_test, y_train, y_test = train_test_split(clean_data,label,test_size=1/7)
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=1/6)

In [35]:
class PrejudiceRemover(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, inputs, targets):
        N = inputs.shape[0]
        N_s1 = inputs[inputs[:,0]==1].shape[0]
        N_s0 = inputs[inputs[:,0]==0].shape[0]
        
        P_ys1 = torch.sum(model(inputs[inputs[:,0]==1])) / N_s1
        P_ys0 = torch.sum(model(inputs[inputs[:,0]==0])) / N_s0
        P_y = torch.sum(model(inputs)) / N
        
        P_y1s1 = torch.log(P_ys1) - torch.log(P_y)
        P_y0s1 = torch.log(1-P_ys1) - torch.log(1-P_y)
        P_y1s0 = torch.log(P_ys0) - torch.log(P_y)
        P_y0s0 = torch.log(1-P_ys0) - torch.log(1-P_y)
        
        PI_y1s1 = torch.sum(model(inputs[inputs[:,0]==1]) * P_y1s1)
        PI_y0s1 = torch.sum((1- model(inputs[inputs[:,0]==1])) * P_y0s1)
        PI_y1s0 = torch.sum(model(inputs[inputs[:,0]==0]) * P_y1s0)
        PI_y0s0 = torch.sum((1- model(inputs[inputs[:,0]==0]))* P_y0s0)
        
        PI = PI_y1s1 + PI_y0s1 + PI_y1s0 + PI_y0s0
        return PI
        
pr = PrejudiceRemover

In [36]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs

epochs = 1000
input_dim = 420 
output_dim = 1 
learning_rate = 0.01

model = LogisticRegression(input_dim,output_dim)

criterion = nn.BCELoss(reduction='sum')

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

X_train, X_test = torch.Tensor(np.array(X_train)),torch.Tensor(np.array(X_test))
y_train, y_test = torch.Tensor(np.array(y_train)),torch.Tensor(np.array(y_test))

losses = []
losses_test = []
Iterations = []
iter = 0
for epoch in range(epochs):
    x = X_train
    labels = y_train
    optimizer.zero_grad() 
    outputs = model(X_train)
    loss = criterion(torch.squeeze(outputs), torch.tensor(np.array(labels)))+pr.forward(pr,X_train,y_train)
    loss.backward()  
    optimizer.step() 
    
    iter+=1
    if iter%100==0:
        # calculate Accuracy
        with torch.no_grad():
            # Calculating the loss and accuracy for the test dataset
            correct_test = 0
            total_test = 0
            outputs_test = torch.squeeze(model(X_test))
            loss_test = criterion(outputs_test, torch.tensor(np.array(y_test)))+pr.forward(pr,X_test,y_test)
            print(pr.forward(pr,X_test,y_test))
            
            predicted_test = outputs_test.round().detach().numpy()
            total_test += y_test.size(0)
            correct_test += np.sum(predicted_test == y_test.detach().numpy())
            accuracy_test = 100 * correct_test/total_test
            losses_test.append(loss_test.item())
            print(pr.forward(pr,X_test,y_test))
            
            # Calculating the loss and accuracy for the train dataset
            total = 0
            correct = 0
            total += y_train.size(0)
            correct += np.sum(torch.squeeze(outputs).round().detach().numpy() == y_train.detach().numpy())
            accuracy = 100 * correct/total
            losses.append(loss.item())
            Iterations.append(iter)
            
            print(f"Iteration: {iter}. \nTest - Loss: {loss_test.item()}. Accuracy: {accuracy_test}")
            print(f"Train -  Loss: {loss.item()}. Accuracy: {accuracy}\n")

tensor(5.7766)
tensor(5.7766)
Iteration: 100. 
Test - Loss: 523.0494384765625. Accuracy: 90.6392694063927
Train -  Loss: 1605.1719970703125. Accuracy: 91.24971441626685

tensor(5.6252)
tensor(5.6252)
Iteration: 200. 
Test - Loss: 512.8511352539062. Accuracy: 90.75342465753425
Train -  Loss: 1567.66162109375. Accuracy: 91.18117432031072

tensor(5.6147)
tensor(5.6147)
Iteration: 300. 
Test - Loss: 587.6864624023438. Accuracy: 90.98173515981735
Train -  Loss: 1541.3040771484375. Accuracy: 91.2725611149189

tensor(5.6582)
tensor(5.6582)
Iteration: 400. 
Test - Loss: 580.1775512695312. Accuracy: 91.0958904109589
Train -  Loss: 1518.2100830078125. Accuracy: 91.22686771761481

tensor(5.7051)
tensor(5.7051)
Iteration: 500. 
Test - Loss: 573.94287109375. Accuracy: 91.0958904109589
Train -  Loss: 1496.5677490234375. Accuracy: 91.24971441626685

tensor(5.7347)
tensor(5.7347)
Iteration: 600. 
Test - Loss: 568.8789672851562. Accuracy: 91.0958904109589
Train -  Loss: 1475.14794921875. Accuracy: 91.3